# U-net coding structure

The coding structure of our design. has five imprtant code parts. There are transforms, data loader, subsample, model and training.

## Transforms

- The transforms part is consist of some functions used in this neural network. For example, fft2() and ifft2() are Fourier and inverse Fourier functions that are very critical. The two functions are used to process the k-sapce data and images.

## Data Loader

- This code first loads all file names, paths and slices by using the load_data_path method. Specifically, in load_data_path method, the first 5 slices are mostly noise so it is better to exclude them. And then, we set some parameters in order to set MRI data, including acceleration, central fraction, seed, number of batch and workers. The last two parameters decide speed of data loading. After that, a few slices from each volume have been randomly selected. Then, the data loader are created for training set by using function Dataloder. Finally, we stack different slices into a volume for visualization and display the mask, masked k-space, under sampled image and ground truth image on screen.

In [ ]:
def show_slices(data, slice_nums, cmap=None): # visualisation
    fig = plt.figure(figsize=(15,10))
    for i, num in enumerate(slice_nums):
        plt.subplot(1, len(slice_nums), i + 1)
        plt.imshow(data[num], cmap=cmap)
        plt.axis('off')

In [ ]:
class MRIDataset(DataLoader):
    def __init__(self, data_list, acceleration, center_fraction, use_seed):
        self.data_list = data_list # data list imported to be processed
        self.acceleration = acceleration # defined AF
        self.center_fraction = center_fraction # defined CF
        self.use_seed = use_seed # defined random or stable mask

    def __len__(self): # create an attribute of the length of dataset
        return len(self.data_list) # 

    def __getitem__(self, idx): # 
        subject_id = self.data_list[idx]
        return get_epoch_batch(subject_id, self.acceleration, self.center_fraction, self.use_seed)

In [ ]:
def get_epoch_batch(subject_id, acc, center_fract, use_seed=True):
    ''' random select a few slices (batch_size) from each volume'''

    fname, rawdata_name, slice = subject_id  
    
    with h5py.File(rawdata_name, 'r') as data:
        rawdata = data['kspace'][slice]
                      
    slice_kspace = T.to_tensor(rawdata).unsqueeze(0)
    S, Ny, Nx, ps = slice_kspace.shape

    # apply random mask
    shape = np.array(slice_kspace.shape)
    mask_func = MaskFunc(center_fractions=[center_fract], accelerations=[acc])
    seed = None if not use_seed else tuple(map(ord, fname))
    mask = mask_func(shape, seed)
      
    # undersample
    masked_kspace = torch.where(mask == 0, torch.Tensor([0]), slice_kspace)
    masks = mask.repeat(S, Ny, 1, ps)

    img_gt, img_und = T.ifft2(slice_kspace), T.ifft2(masked_kspace)

    # perform data normalization which is important for network to learn useful features
    # during inference there is no ground truth image so use the zero-filled recon to normalize
    norm = T.complex_abs(img_und).max()
    if norm < 1e-6: norm = 1e-6
    
    # normalized data
    img_gt, img_und, rawdata_und = img_gt/norm, img_und/norm, masked_kspace/norm
        
    return img_gt.squeeze(0), img_und.squeeze(0), rawdata_und.squeeze(0), masks.squeeze(0), norm

In [ ]:
def load_data_path(train_data_path, val_data_path):
    """ Go through each subset (training, validation) and list all 
    the file names, the file paths and the slices of subjects in the training and validation sets 
    """

    data_list = {} # initiate the data list
    train_and_val = ['train', 'val'] # set the keys in data list
    data_path = [train_data_path, val_data_path] # import the data path for both training and validation
      
    for i in range(len(data_path)): # iterate on both paths

        data_list[train_and_val[i]] = [] # reset the data list 
        
        which_data_path = data_path[i] # set the data path to current dataset
    
        for fname in sorted(os.listdir(which_data_path)): # iterate on all listed files in the data path
            
            subject_data_path = os.path.join(which_data_path, fname) # set subject data path with its directory
                     
            if not os.path.isfile(subject_data_path): continue # iterate until all files are registered on path
            
            with h5py.File(subject_data_path, 'r') as data: # open the subject at the desinated path
                num_slice = data['kspace'].shape[0] # set # of slices as the 1st-D of the data's kspace data
                
            # the first 5 slices are mostly noise so it is better to exlude them
            data_list[train_and_val[i]] += [(fname, subject_data_path, slice) for slice in range(5, num_slice)]
            # data_list is concatenated by a list each consisted with a file's name, path and slices excluding 
            # the first 5 slices that are  mostly considered as noise 
            
    return data_list # {'train': [('file1000000.h5','/home/kevinxu/Documents/NC2019MRI/train/file1000000.h5',5)？

In [ ]:
if __name__ == '__main__': # main() function of the file
    
    data_path_train = '/home/kevinxu/Documents/NC2019MRI/train'
    data_path_val = '/home/kevinxu/Documents/NC2019MRI/train'
    data_list = load_data_path(data_path_train, data_path_val) # first load all file names, paths and slices.

    acc = 8 # AF
    cen_fract = 0.04 # central fraction that are set to be included
    seed = False # random masks for each slice  用来随机生成mask
    num_batch = 1 # batch size of each loading process 每次处理一张图片
    num_workers = 12 # data loading is faster using a bigger number for num_workers. 0 means using cpu. 多线程处理多张图片
    
    # create data loader for training set. It applies same to validation set as well
    train_dataset = MRIDataset(data_list['train'], acceleration=acc, center_fraction=cen_fract, use_seed=seed)
    train_loader = DataLoader(train_dataset, shuffle=True, batch_size=num_batch, num_workers=num_workers) 
    for iteration, sample in enumerate(train_loader):
        
        img_gt, img_und, rawdata_und, masks, norm = sample
         
        # stack different slices into a volume for visualisation
        A = masks[...,0].squeeze()    # mask applied
        B = torch.log(T.complex_abs(rawdata_und) + 1e-9).squeeze()    # masked kspace image
        C = T.complex_abs(img_und).squeeze()    # image generated from under-sampled data
        D = T.complex_abs(img_gt).squeeze()    # image generated from ground truth data
        all_imgs = torch.stack([A,B,C,D], dim=0)

        # from left to right: mask, masked kspace, undersampled image, ground truth
        show_slices(all_imgs, [0, 1, 2, 3], cmap='gray')
        plt.pause(0.1)

        if iteration >= 3: break  # show 4 random slices

## Subsample

- Subsample is used to apply random mask for k-space data.

In [ ]:
class MaskFunc:

    def __init__(self, center_fractions, accelerations):
       
        if len(center_fractions) != len(accelerations):
            raise ValueError('Number of center fractions should match number of accelerations')

        self.center_fractions = center_fractions
        self.accelerations = accelerations
        self.rng = np.random.RandomState()

    def __call__(self, shape, seed=None):

        if len(shape) < 3:
            raise ValueError('Shape should have 3 or more dimensions')

        self.rng.seed(seed)
        num_cols = shape[-2]

        choice = self.rng.randint(0, len(self.accelerations))
        center_fraction = self.center_fractions[choice]
        acceleration = self.accelerations[choice]

        num_low_freqs = int(round(num_cols * center_fraction))
        prob = (num_cols / acceleration - num_low_freqs) / (num_cols - num_low_freqs)
        mask = self.rng.uniform(size=num_cols) < prob
        pad = (num_cols - num_low_freqs + 1) // 2
        mask[pad:pad + num_low_freqs] = True

        mask_shape = [1 for _ in shape]
        mask_shape[-2] = num_cols
        mask = torch.from_numpy(mask.reshape(*mask_shape).astype(np.float32))

        return mask

## Model and training

 - The U-NET model has been described in the design part. So, in this part, I decide show some details. The channels of input images and output images are set one, and the kernels' channels are 64 which means after we did the stride-2 convolutional operation, leading to 64 channels images. Because the value of drop probability is 0.5，each layer will randomly drop approximate 50 percent cells. Besides, experimenters set the value of number of pool layers is 3.Three pool layers means the ConvBlock will be operated three times in down sampling and twice in up sampling.
 - In the training part, we choose the L1 loss function. In addition, we adopt the Stochastic Gradient Decent(SGD) optimizer which is the most basic optimization method and common training methods. Then, the value of learning rate we set is 0.01.Finally, we can train the data by forward propagation and back propagation.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = UnetModel(in_chans=1, out_chans=1, chans=64, num_pool_layers=3, drop_prob=0.5).to(device)
loss = torch.nn.L1Loss()
optimizer = optim.SGD(params=model.parameters(), lr=0.01)

def train(epoch):
    model.train()
    for i, sample in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad() #initialize gradient to zero
        output = model(data) #Finding the predicted value using forward propagation
        loss.backward() #Back propagation to find the gradient
        optimizer.step() #Update all parameters